In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
path = '/content/drive/My Drive/kaggle/HM/'
os.chdir(path) # カレントディレクトリを指定
os.getcwd()

'/content/drive/My Drive/kaggle/HM'

In [3]:
!ls ./recbox_data

recbox_data_big.inter  recbox_data.inter  recbox_data.item  recbox_data.user


In [4]:
!pip install recbole

# Import

In [21]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4RecF, BERT4Rec, repeatnet, SRGNN, npe, hrm, narm, shan
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

import numpy as np
import pandas as pd
from tqdm import tqdm

In [22]:
parameter_dict = {
    'data_path': './',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp'],
                 'item': ['item_id', 'product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no',
                      'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                      'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'season', 'price_mean', 'price_max', 'mean_sales_channel_id', 'min_timestamp'],
                 'user': ['user_id', 'on_offline', 'price_level', 'children_count', 'ladieswear_count', 'menswear', 'active', 'age']
             },
    'selected_features': ['product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no',
                          'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                          'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'season', 'price_mean', 'price_max', 'mean_sales_channel_id', 'min_timestamp'],
    'neg_sampling': None,
    'epochs': 70,
    'eval_args': {
        'split': {'RS': [10, 0, 0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'},
}

config = Config(model='GRU4RecF', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

09 May 15:43    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ./recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 70
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = None
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = timestamp
seq_len

In [23]:
dataset = create_dataset(config)
logger.info(dataset)

09 May 15:45    INFO  recbox_data
The number of users: 15459
Average actions of users: 59.21956268598784
The number of items: 7330
Average actions of items: 124.9032610178742
The number of inters: 915416
The sparsity of the dataset: 99.19214553975321%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'active', 'age', 'on_offline', 'price_level', 'children_count', 'ladieswear_count', 'menswear', 'product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'price_mean', 'price_max', 'mean_sales_channel_id', 'season', 'min_timestamp']
recbox_data
The number of users: 15459
Average actions of users: 59.21956268598784
The number of items: 7330
Average actions of items: 124.9032610178742
The number of inters: 915416
The sparsity of the dataset: 99.19214553975321%
Remain Fields: ['user_id', 'item_id', 'timest

In [24]:
train_data, valid_data, test_data = data_preparation(config, dataset)
# model loading and initialization
model = GRU4RecF(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

09 May 15:45    INFO  [Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
09 May 15:45    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
09 May 15:45    INFO  GRU4RecF(
  (item_embedding): Embedding(7330, 64, padding_idx=0)
  (feature_embed_layer): FeatureSeqEmbLayer(
    (token_embedding_table): ModuleD

In [25]:
# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

09 May 15:46    INFO  epoch 0 training [time: 74.34s, train loss: 3687.0453]
epoch 0 training [time: 74.34s, train loss: 3687.0453]
epoch 0 training [time: 74.34s, train loss: 3687.0453]
epoch 0 training [time: 74.34s, train loss: 3687.0453]
09 May 15:46    INFO  Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
09 May 15:48    INFO  epoch 1 training [time: 77.13s, train loss: 3478.8041]
epoch 1 training [time: 77.13s, train loss: 3478.8041]
epoch 1 training [time: 77.13s, train loss: 3478.8041]
epoch 1 training [time: 77.13s, train loss: 3478.8041]
09 May 15:48    INFO  Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
Saving current: saved/GRU4RecF-May-09-2022_15-45-32.pth
09 May 15:49    INFO  ep

# predict

In [26]:
!ls ./recbox_data/

recbox_data_big.inter  recbox_data.inter  recbox_data.item  recbox_data.user


In [27]:
# from recbole.quick_start.quick_start import load_data_and_model

# config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
#     model_file='saved/GRU4RecF-May-08-2022_15-16-33.pth')

In [28]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [29]:
len(external_user_ids)

15458

In [30]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 12)

In [31]:
predict_for_all_item('0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef', 
                     dataset, model)

torch.return_types.topk(values=tensor([[8.4764, 7.6985, 7.2913, 6.8105, 6.6141, 6.5500, 6.2963, 6.2728, 6.1344,
         6.0105, 5.9968, 5.9503]], device='cuda:0'), indices=tensor([[4559, 4529, 5835, 4580, 1608, 2159, 6691, 5695, 5824, 6730, 4579, 6483]],
       device='cuda:0'))

In [32]:
topk_items = []
for external_user_id in tqdm(external_user_ids):
  try:
    _, topk_iid_list = predict_for_all_item(external_user_id, dataset, model)
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
  except IndexError:
    pass

print(len(topk_items))

100%|██████████| 15458/15458 [02:43<00:00, 94.74it/s]


15458


In [33]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['customer_id'])
result['prediction'] = external_item_str
result.head()

,customer_id,prediction
0,00d7ebd46f6a6d53630d41386b6ef6a505cdc4c80011ff...,0918522001 0915526001 0881942002 0914805002 08...
1,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0901955001 0861477001 0913030001 0776237031 08...
2,013f00f9e218549246a3aa82b3f3a0c22a693bc25fa735...,0839402002 0857621002 0857621001 0843284001 08...
3,01bada2a453b09c70ea57bdda5a9ef0fb04062718d3a3d...,0724906006 0914441001 0723469001 0820572001 09...
4,01dd96059a11759518f10969d0a528f03c8501dc4c628b...,0859118001 0910872001 0876053003 0817353003 08...


In [34]:
result.to_csv('./output/result.csv', index = False)